In [1]:
import random

''' 
    1. Read from 'movie-lines.txt'
    2. Create a dictionary with ( key = line_id, value = text )
'''
def get_id2line():
    lines=open('data/cornell movie-dialogs corpus/movie_lines.txt', "r",encoding='utf-8', errors='ignore').read().split('\n')
    id2line = {}
    for line in lines:
        _line = line.split(' +++$+++ ')
        if len(_line) == 5:
            id2line[_line[0]] = _line[4]
    return id2line

'''
    1. Read from 'movie_conversations.txt'
    2. Create a list of [list of line_id's]
'''
def get_conversations():
    conv_lines = open('data/cornell movie-dialogs corpus/movie_conversations.txt', "r",encoding='utf-8', errors='ignore').read().split('\n')
    convs = [ ]
    for line in conv_lines[:-1]:
        _line = line.split(' +++$+++ ')[-1][1:-1].replace("'","").replace(" ","")
        convs.append(_line.split(','))
    return convs

'''
    1. Get each conversation
    2. Get each line from conversation
    3. Save each conversation to file
'''
def extract_conversations(convs,id2line,path=''):
    idx = 0
    for conv in convs:
        f_conv = open(path + str(idx)+'.txt', 'w')
        for line_id in conv:
            f_conv.write(id2line[line_id])
            f_conv.write('\n')
        f_conv.close()
        idx += 1

'''
    Get lists of all conversations as Questions and Answers
    1. [questions]
    2. [answers]
'''
def gather_dataset(convs, id2line):
    questions = []; answers = []

    for conv in convs:
        if len(conv) %2 != 0:
            conv = conv[:-1]
        for i in range(len(conv)):
            if i%2 == 0:
                questions.append(id2line[conv[i]])
            else:
                answers.append(id2line[conv[i]])

    return questions, answers


'''
    We need 4 files
    1. train.enc : Encoder input for training
    2. train.dec : Decoder input for training
    3. test.enc  : Encoder input for testing
    4. test.dec  : Decoder input for testing
'''

'\n    We need 4 files\n    1. train.enc : Encoder input for training\n    2. train.dec : Decoder input for training\n    3. test.enc  : Encoder input for testing\n    4. test.dec  : Decoder input for testing\n'

In [2]:
import numpy as np

id2line = get_id2line()
convs = get_conversations()
questions, answers = gather_dataset(convs, id2line)
questions = np.array(questions)
answers = np.array(answers)

In [3]:
print(len(questions))
print(len(answers))

138135
138135


In [11]:
%%time
from hedgeable_ai.functions.preprocessing.word2index import Word2IndexProcessor
from hedgeable_ai.functions.preprocessing.clean_text import clean_text

N = 10000000000
texts = list(questions[:N]) + list(answers[:N])
processor = Word2IndexProcessor(texts, tokenizer=clean_text)

CPU times: user 1h 8min 54s, sys: 1min 5s, total: 1h 10min
Wall time: 1h 9min 14s


In [12]:
len(texts)

276270

In [13]:
from __future__ import print_function
import tensorflow as tf

def layer_normalization(inputs, epsilon=1e-8, scope="layer_normalization", reuse=None):
    '''Applies layer normalization.
    
    Args:
      inputs: A tensor with 2 or more dimensions, where the first dimension has
        `batch_size`.
      epsilon: A floating number. A very small number for preventing ZeroDivision Error.
      scope: Optional scope for `variable_scope`.
      reuse: Boolean, whether to reuse the weights of a previous layer
        by the same name.
      
    Returns:
      A tensor with the same shape and data dtype as `inputs`.
    '''
    with tf.variable_scope(scope, reuse=reuse):
        shape = inputs.get_shape().as_list()
        hidden_dim = shape[-1]
        mean, variance = tf.nn.moments(inputs, [-1], keep_dims=True)
        gamma = tf.get_variable("gamma", hidden_dim, initializer=tf.ones_initializer(tf.float32))
        beta = tf.get_variable("beta", hidden_dim, initializer=tf.zeros_initializer(tf.float32))
        normalized_inputs = (inputs - mean) / tf.sqrt(variance + epsilon)
        outputs = gamma * normalized_inputs + beta
    return outputs

def embedding(inputs,vocab_size, num_units, 
              zero_pad=True, scale=True,
              scope="embedding", reuse=None):
    '''Embeds a given tensor.
    Args:
      inputs: A `Tensor` with type `int32` or `int64` containing the ids
         to be looked up in `lookup table`.
      vocab_size: An int. Vocabulary size.
      num_units: An int. Number of embedding hidden units.
      zero_pad: A boolean. If True, all the values of the fist row (id 0)
        should be constant zeros.
      scale: A boolean. If True. the outputs is multiplied by sqrt num_units.
      scope: Optional scope for `variable_scope`.
      reuse: Boolean, whether to reuse the weights of a previous layer
        by the same name.
    Returns:
      A `Tensor` with one more rank than inputs's. The last dimensionality
        should be `num_units`.
        
    For example,
    
    ```
    import tensorflow as tf
    
    inputs = tf.to_int32(tf.reshape(tf.range(2*3), (2, 3)))
    outputs = embedding(inputs, 6, 2, zero_pad=True)
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        print sess.run(outputs)
    >>
    [[[ 0.          0.        ]
      [ 0.09754146  0.67385566]
      [ 0.37864095 -0.35689294]]
     [[-1.01329422 -1.09939694]
      [ 0.7521342   0.38203377]
      [-0.04973143 -0.06210355]]]
    ```
    
    ```
    import tensorflow as tf
    
    inputs = tf.to_int32(tf.reshape(tf.range(2*3), (2, 3)))
    outputs = embedding(inputs, 6, 2, zero_pad=False)
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        print sess.run(outputs)
    >>
    [[[-0.19172323 -0.39159766]
      [-0.43212751 -0.66207761]
      [ 1.03452027 -0.26704335]]
     [[-0.11634696 -0.35983452]
      [ 0.50208133  0.53509563]
      [ 1.22204471 -0.96587461]]]    
    ```    
    '''
    with tf.variable_scope(scope, reuse=reuse):
        lookup_table = tf.get_variable('lookup_table',
                                       dtype=tf.float32,
                                       shape=[vocab_size, num_units],
                                       initializer=tf.contrib.layers.xavier_initializer())
        if zero_pad:
            lookup_table = tf.concat((tf.zeros(shape=[1, num_units]),
                                      lookup_table[1:, :]), 0)
        outputs = tf.nn.embedding_lookup(lookup_table, inputs)
        
        if scale:
            outputs = outputs * tf.sqrt(num_units) 
            
    return outputs
    
def multihead_attention(queries, 
                        keys,
                        values,
                        values_input=None, 
                        num_units=None, 
                        num_heads=8, 
                        drop_rate=0,
                        training=True,
                        causality=False,
                        scope="multihead_attention", 
                        reuse=None):
    '''Applies multihead attention.
    
    Args:
      queries: A 3d tensor with shape of [N, T_q, C_q].
      keys: A 3d tensor with shape of [N, T_k, C_k].
      values: A 3d tensor with shape of [N, T_k, C_k].
      num_units: A scalar. Attention size.
      drop_rate: A floating point number.
      training: Boolean. Controller of mechanism for dropout.
      causality: Boolean. If true, units that reference the future are masked. 
      num_heads: An int. Number of heads.
      scope: Optional scope for `variable_scope`.
      reuse: Boolean, whether to reuse the weights of a previous layer
        by the same name.
        
    Returns
      A 3d tensor with shape of (N, T_q, C)  
    '''
    with tf.variable_scope(scope, reuse=reuse):
        # Set the fall back option for num_units
        if num_units is None:
            num_units = queries.get_shape().as_list()[-1]
        
        # Linear projections
        Q = tf.layers.dense(queries, num_units, activation=None) # (N, T_q, C)
        K = tf.layers.dense(keys, num_units, activation=None) # (N, T_k, C)
        V = tf.layers.dense(values, num_units, activation=None) # (N, T_k, C)
        
        # Split and concat
        # The size will be [N * num_heads, T_k, C_k/num_heads]
        Q_ = tf.concat(tf.split(Q, num_heads, axis=2), axis=0) # (h*N, T_q, C/h) 
        K_ = tf.concat(tf.split(K, num_heads, axis=2), axis=0) # (h*N, T_k, C/h) 
        V_ = tf.concat(tf.split(V, num_heads, axis=2), axis=0) # (h*N, T_k, C/h) 

        # Multiplication
        alignments = tf.matmul(Q_, tf.transpose(K_, [0, 2, 1])) # (h*N, T_q, T_k)
  
        # Causality = Future blinding
        if causality:
            shape = tf.shape(alignments)
            masks = tf.ones(shape[1:]) # (T_q, T_k)
            masks = tf.contrib.linalg.LinearOperatorTriL(masks).to_dense() # (T_q, T_k)
            masks = tf.tile(tf.expand_dims(masks, 0), [shape[0], 1, 1]) # (h*N, T_q, T_k)
            paddings = tf.ones_like(masks)*(-2**32+1) # minimum value for float
            alignments = tf.where(tf.equal(masks, 0), paddings, alignments) # (h*N, T_q, T_k)
        # avoid zeropadding output alignments
        if values_input is not None:
            T_query = tf.shape(alignments)[1]
            input_ = tf.concat([values_input for _ in range(num_heads)], axis=0)
            input_sign = tf.expand_dims(tf.sign(input_), 1)
            input_sign = tf.cast(tf.tile(input_sign, [1, T_query, 1]), tf.int32)
            paddings = tf.ones_like(input_sign, dtype=tf.float32)*(-2**32+1)
            alignments = tf.where(tf.equal(input_sign, 0), paddings, alignments)
        # Scale
        d = tf.constant(K_.get_shape().as_list()[-1], tf.float32)
        alignments = tf.nn.softmax(alignments / tf.sqrt(d))
        
        # Dropouts
        alignments = tf.layers.dropout(alignments, rate=drop_rate, training=training)
        # Weighted sum
        outputs = tf.matmul(alignments, V_) # ( h*N, T_q, C/h)
        # Restore shape
        outputs = tf.concat(tf.split(outputs, num_heads, axis=0), axis=2) # (N, T_q, C)
        # Residual connection
        outputs += queries 
        # Normalize
        outputs = layer_normalization(outputs) # (N, T_q, C)
    return outputs

def feedforward(inputs, 
                num_units=[2048, 512],
                scope="multihead_attention", 
                reuse=None):
    '''Point-wise feed forward net.
    
    Args:
      inputs: A 3d tensor with shape of [N, T, C].
      num_units: A list of two integers.
      scope: Optional scope for `variable_scope`.
      reuse: Boolean, whether to reuse the weights of a previous layer
        by the same name.
        
    Returns:
      A 3d tensor with the same shape and dtype as inputs
    '''
    with tf.variable_scope(scope, reuse=reuse):
        # Inner layer
        params = {"inputs": inputs, "filters": num_units[0], "kernel_size": 1,
                  "activation": tf.nn.relu, "use_bias": True}
        outputs = tf.layers.conv1d(**params)
        
        # Readout layer
        params = {"inputs": outputs, "filters": num_units[1], "kernel_size": 1,
                  "activation": None, "use_bias": True}
        outputs = tf.layers.conv1d(**params)
        
        # Residual connection
        outputs += inputs
        
        # Normalize
        outputs = layer_normalization(outputs)
    
    return outputs

def label_smoothing(inputs, epsilon=0.1):
    '''Applies label smoothing. See https://arxiv.org/abs/1512.00567.
    
    Args:
      inputs: A 3d tensor with shape of [N, T, V], where V is the number of vocabulary.
      epsilon: Smoothing rate.
    
    For example,
    
    ```
    import tensorflow as tf
    inputs = tf.convert_to_tensor([[[0, 0, 1], 
       [0, 1, 0],
       [1, 0, 0]],
      [[1, 0, 0],
       [1, 0, 0],
       [0, 1, 0]]], tf.float32)
       
    outputs = label_smoothing(inputs)
    
    with tf.Session() as sess:
        print(sess.run([outputs]))
    
    >>
    [array([[[ 0.03333334,  0.03333334,  0.93333334],
        [ 0.03333334,  0.93333334,  0.03333334],
        [ 0.93333334,  0.03333334,  0.03333334]],
       [[ 0.93333334,  0.03333334,  0.03333334],
        [ 0.93333334,  0.03333334,  0.03333334],
        [ 0.03333334,  0.93333334,  0.03333334]]], dtype=float32)]   
    ```    
    '''
    K = inputs.get_shape().as_list()[-1] # number of channels
    return ((1-epsilon) * inputs) + (epsilon / K)

In [14]:
import tensorflow as tf
import tensorflow.contrib.rnn as rnn
import tensorflow.contrib.seq2seq as seq2seq
from hedgeable_ai.models.nn import BaseModel, get_shape, get_length
import tensorflow.contrib.rnn as rnn

import tensorflow as tf

from hedgeable_ai.models.nn.params import nn_is_logit
from hedgeable_ai.models.nn import BaseNN, get_shape

from hedgeable_ai.models.nn.rnn import get_cell

count = 0
class DialogueAgent(BaseNN):
    def __init__(self, processor, maxlen=20,
                 conf=None, additional_length=3,
                 num_hidden=512, num_heads=8, num_blocks=6, drop_rate=0.1,
                 position_scale=1000, *args, **kwargs):
        self.num_blocks = num_blocks
        # leave index 0 for padding and 1 for  <eos>
        self.vocab_size = processor.vocab_size
        self.maxlen = maxlen
        self.additional_length = additional_length
        self.dec_maxlen = maxlen + additional_length
        self.position_scale = position_scale
        self.drop_rate = drop_rate
        self.num_heads = num_heads
        self.num_hidden = num_hidden
        self.reuse = False
        super().__init__(processor=processor, conf=conf, *args, **kwargs)
        
    def _build_graph(self):
        """Build tensorflow graph
        
        Note:
            You build graphs for output and input, which will be used 
            for training and prediction.
        """
        # Build Basic Netwoiork
        self.enc_input = tf.placeholder(tf.int32, shape=(None, None), name="encoder_input")
        self.dec_input = tf.placeholder(tf.int32, shape=(None, None), name="decoder_input")
        batch_size = tf.shape(self.dec_input)[0]
        # prepare input data for decoder
        clean_zero_padding = tf.zeros_like(self.dec_input)
        cleaned_dec_input = tf.where(self.dec_input <=1, clean_zero_padding, self.dec_input)
        eos_padding = tf.ones((batch_size, 1), dtype=tf.int32)
        _dec_input = tf.concat((eos_padding, cleaned_dec_input), axis=1)
        # prepare target data for decoder
        zero_padding = tf.zeros((batch_size, 1), dtype=tf.int32)
        dec_target = tf.concat((self.dec_input, zero_padding), axis=1)
        
        self._dec_input = _dec_input
        self.dec_target = dec_target
        logits = self._get_output(self.enc_input, _dec_input, self.training)
        # predictions = tf.cast(tf.arg_max(self.logits, dimension=-1), tf.int32)
        target_smoothed = label_smoothing(tf.one_hot(dec_target, depth=self.vocab_size), epsilon=0.1)
        _loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=target_smoothed)
        masks = tf.cast(tf.sign(dec_target), tf.float32)
        self.loss = tf.reduce_mean(tf.reduce_sum(masks * _loss, [1]))
        
        self.learning_rate_op = self._get_learning_rate()
        update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
        # with tf.control_dependencies(update_ops):
        self.optimizer = self._get_optimizer(self.optimizer_name, self.learning_rate_op, self.optimizer_conf)
        grads_vars = self.optimizer.compute_gradients(self.loss)
        if "grad_clip" in self.conf and self.conf["grad_clip"] is not None:
            grads_vars = [
                (tf.clip_by_norm(gv[0], clip_norm=self.conf["grad_clip"]), gv[1]) 
                    for gv in grads_vars]
        self.train_step = self.optimizer.apply_gradients(grads_vars)
        # prediction flow
        # logits = self._get_output(self.enc_input, _dec_input, self.training)
        self.predictions = tf.cast(tf.arg_max(logits, dimension=-1), tf.int32)
        self.logits = logits
        
    def _get_output(self, enc_input, dec_input, training=True):
        tensor_batch_size = tf.shape(enc_input)[0]
        with tf.variable_scope("embedding", reuse=self.reuse):
            embeddings = tf.get_variable("embedding", [self.vocab_size, self.num_hidden],
                                     initializer=tf.contrib.layers.xavier_initializer())
            embeddings = tf.sqrt(float(self.num_hidden)) * embeddings
        # Encoder
        with tf.variable_scope("encoder", reuse=self.reuse):
            enc_input_embedded = tf.nn.embedding_lookup(embeddings, self.enc_input)
            ## Positional Encoding
            enc_length = tf.shape(enc_input)[1]
            position_idx = tf.range(tf.reduce_max(enc_length))
            position_idx = tf.tile(tf.expand_dims(position_idx, 0), [tensor_batch_size, 1])
            enc_position_embeddings = tf.get_variable("embedding_position",
                                                      [self.maxlen, self.num_hidden],
                                                      initializer=tf.contrib.layers.xavier_initializer())
            enc_position_embedded = tf.nn.embedding_lookup(enc_position_embeddings, position_idx)
            enc_x = enc_input_embedded + enc_position_embedded
            ## Dropout
            enc_x = tf.layers.dropout(enc_x, rate=self.drop_rate, training=training)
            # Encoder blocks    
            for i in range(self.num_blocks):
                with tf.variable_scope("num_blocks_{}".format(i)):
                    ### Multihead Attention
                    enc_x = multihead_attention(queries=enc_x, 
                                                keys=enc_x, 
                                                values=enc_x,
                                                values_input=enc_input,
                                                num_units=self.num_hidden,
                                                num_heads=self.num_heads, 
                                                drop_rate=self.drop_rate,
                                                training=training,
                                                causality=False,
                                                reuse=self.reuse,
                                                scope="self_attention")
                    ### Feed Forward
                    enc_x = feedforward(enc_x, num_units=[4*self.num_hidden, self.num_hidden])
            
        # Decoder
        with tf.variable_scope("decoder", reuse=self.reuse):
            dec_input_embedded = tf.nn.embedding_lookup(embeddings, dec_input)
            ## Positional Encoding
            dec_length = tf.shape(dec_input)[1]
            position_idx = tf.range(dec_length)
            position_idx = tf.tile(tf.expand_dims(position_idx, 0), [tensor_batch_size, 1])
            dec_position_embeddings = tf.get_variable("embedding_position",
                                                      [self.dec_maxlen, self.num_hidden],
                                                      initializer=tf.contrib.layers.xavier_initializer())
            dec_position_embedded = tf.nn.embedding_lookup(dec_position_embeddings, position_idx)
            dec_x = dec_input_embedded + dec_position_embedded
            ## Dropout
            dec_x = tf.layers.dropout(dec_x, rate=self.drop_rate, training=training)
                
            ## Blocks
            for i in range(self.num_blocks):
                with tf.variable_scope("num_blocks_{}".format(i)):
                    ### Multihead Attention
                    dec_x = multihead_attention(queries=dec_x, 
                                                keys=dec_x, 
                                                values=dec_x,
                                                values_input=dec_input,
                                                num_units=self.num_hidden,
                                                num_heads=self.num_heads, 
                                                drop_rate=self.drop_rate,
                                                training=training,
                                                causality=True,
                                                reuse=self.reuse,
                                                scope="self_attention")
                        
                    dec_x = multihead_attention(queries=dec_x, 
                                                keys=enc_x, 
                                                values=enc_x,
                                                values_input=enc_input,
                                                num_units=self.num_hidden,
                                                num_heads=self.num_heads, 
                                                drop_rate=self.drop_rate,
                                                training=training,
                                                causality=False,
                                                reuse=self.reuse,
                                                scope="vanilla_attention")
                    ### Feed Forward
                    dec_x = feedforward(dec_x, num_units=[4*self.num_hidden, self.num_hidden],
                                       scope="fully_connected", reuse=self.reuse)
                
            # Final linear projection
            dim_size = dec_x.get_shape().as_list()[2]
            shape = tf.shape(dec_x)
            dec_x = tf.reshape(dec_x, [-1, shape[2]])
            logits = tf.matmul(dec_x, tf.transpose(embeddings))
            logits = tf.reshape(logits, [shape[0], shape[1], self.vocab_size])
        self.reuse = True
        return logits
        
    def _optimize(self, batch_X, batch_y, *args, **kwargs):
        global count
        batch_X = batch_X[0]
        batch_X, Xlen = self.processor.batch_padding(batch_X, self.maxlen)
        # print("batch_X", [self.processor.decode(i) for i in batch_X])
        length = np.max(Xlen) + self.additional_length
        batch_y = self._batch_padding(batch_y, length)
        # print("batch_y", [self.processor.decode(i) for i in batch_y])
        feed_dict = {self.enc_input: batch_X,
                     self.dec_input: batch_y,
                     self.training: True}
        
        _, loss = self.sess.run([self.train_step, self.loss], feed_dict=feed_dict)
        if count % 100 == 0:
            idx = np.arange(len(texts))
            np.random.shuffle(idx)
            sentences = np.array(texts)[idx[:5]]
            predictions = self.generate_sentences(sentences)
            for prediction, sentence in zip(predictions, sentences):
                print("question")
                print(sentence)
                print("answer")
                print(prediction)
            print("******************************************")
        count += 1
        return loss
    
    def generate_sentences(self, sentences):
        X = [self.processor.encode(sentence) for sentence in sentences]
        X, Xlen = self.processor.batch_padding(X, self.maxlen)
        batch_size = X.shape[0]
        y = np.array([[] for _ in range(batch_size)], dtype=int)
        not_finished = np.array([True for _ in range(batch_size)])
        for i in range(self.dec_maxlen):
            feed_dict = {self.enc_input: X,
                         self.dec_input: y,
                         self.training: False}
            dec_i, dec_t = self.sess.run([self._dec_input, self.dec_target], feed_dict=feed_dict)
            predictions = self.sess.run(self.predictions, feed_dict=feed_dict)
            new_y = predictions[:, -1]
            new_y = np.array([new_y[k] if not_finished[k] else 0 for k in range(batch_size)])
            y = np.concatenate((y, new_y[:, np.newaxis]), axis=1)
            new_not_finished = np.array(new_y > 1, dtype=int)
            not_finished = np.array(
                np.array(not_finished, dtype=int) * np.array(new_not_finished, dtype=int),
                dtype=bool)
            if np.sum(not_finished) == 0:
                break
        return [self.processor.decode(i) for i in y]
    
    def _batch_padding(self, batch, length):
        PAD = 0
        EOS = 1
        padded_batch = []
        for x in batch:
            x = list(x)
            if len(x) > length:
                x = x[:length]
            if len(x) < length:
                x.append(EOS)
            while len(x) < length:
                x.append(PAD)
            padded_batch.append(x)
        return np.array(padded_batch)

In [15]:
%%time

train_X = processor.batch_process_test(np.array(questions)[:N])
train_y = processor.batch_process_test(np.array(answers)[:N]) 

CPU times: user 2h 4min 44s, sys: 1min 26s, total: 2h 6min 10s
Wall time: 2h 4min 55s


In [16]:
import tensorflow as tf
import numpy as np


conf = {
        "learning_rate": 1e-4,
        "learning_rate_minimum": 1e-4,
        "learning_rate_decay": 0.5,
        "learning_rate_decay_step": 1,
        "batch_size": 64,
        "model_dir": "./attention_logs",
        "load_file_path": None,
        "save_file_path": None,
        "log_freq": 1,
        "grad_clip":None,
        "optimizer":"adam",
}


tf.reset_default_graph()
agent = DialogueAgent(processor, maxlen=20, conf=conf, additional_length=3)
agent.fit(train_X, train_y, num_epochs=10000, batch_bar=False, log_freq=1, batch_log_freq=100)

  0%|          | 0/10000 [00:00<?, ?it/s]

Model saved in file: params/model.ckpt
question
Say you're sick.
answer
['gentlemen', 'gentlemen', 'gentlemen', 'gentlemen', 'gentlemen', 'gentlemen', 'gentlemen', 'gentlemen', 'gentlemen', 'gentlemen', 'gentlemen', 'gentlemen', 'gentlemen', 'gentlemen', 'gentlemen', 'gentlemen', 'gentlemen', 'gentlemen', 'gentlemen', 'gentlemen', 'gentlemen', 'gentlemen', 'gentlemen']
question
Don't you think it meant anything to him. Even if I am fifteen?
answer
['surrey', 'surrey', 'surrey', 'surrey', 'surrey', 'surrey', 'surrey', 'surrey', 'surrey', 'surrey', 'surrey', 'surrey', 'surrey', 'surrey', 'manhood', 'manhood', 'manhood', 'manhood', 'manhood', 'manhood', 'manhood', 'manhood', 'manhood']
question
Hello Paul. Is Telly inside?
answer
['drank', 'drank', 'drank', 'drank', 'drank', 'drank', 'drank', 'drank', 'drank', 'drank', 'drank', 'drank', 'drank', 'drank', 'drank', 'drank', 'drank', 'drank', 'drank', 'drank', 'drank', 'drank', 'drank']
question
I mean 'Nam was a foot soldier's war whereas, 

question
The vaccine you hold is the only defense against the virus. Its introduction into an alien environment may have the power to destroy the delicate plans we've so assiduously protected for the last fifty years.  Or it may not.
answer
['you', 'are', 'not', 'a', 'little']
question
Caiman, wait. Take me with you.
answer
['i', 'am', 'not', 'a', 'little']
question
Where did you hear that?
answer
['i', 'am', 'not', 'a', 'little']
question
Of course, Wes.
answer
['you', 'are', 'not', 'a', 'little']
question
Cool.  I read a book once.  It was like all these people discovering who they used to be.  You know, like reincarnation?  It was really good.  You ever read that?
answer
['you', 'are', 'not', 'a', 'little']
******************************************
question
Why?
answer
['i', 'am', 'not', 'sure']
question
We know there's got to be something or they wouldn't be so panicked.
answer
['i', 'am', 'not', 'sure']
question
Edward, these women here, I don't think any of'm like the guys they'

  0%|          | 1/10000 [08:24<1401:36:43, 504.63s/it]

question
Why? Because of the pain I feel now? I have regrets Will, but I don't regret a singel day I spent with her.
answer
['i', 'do', 'not', 'know']
question
You mind your own bussines.
answer
['i', 'do', 'not', 'know']
question
Such as?
answer
['i', 'am', 'not', 'going', 'to', 'do']
question
This from a girl who still plays with dolls.
answer
['i', 'do', 'not', 'know']
question
Man, that's great.
answer
['i', 'do', 'not', 'know']
******************************************
question
Check you later, man.
answer
['i', 'am', 'sorry']
question
The blood type on the rocks matches Suzie's.
answer
['i', 'do', 'not', 'know']
question
Do you know where she's staying in Hawaii?
answer
['i', 'do', 'not', 'know']
question
Sargent.  Establish a recon post downstairs.  Code red.  You know what to do.
answer
['i', 'do', 'not', 'know']
question
I know you as well as I know myself, Shinzon. There was a time you looked at the stars and dreamed of what might be.
answer
['i', 'do', 'not', 'know']
******

...

question
For a little over two years now we've been spotting serious blowback in IMF operations.  We have a penetration.  The other day we decoded a message on the Internet from a Czech we know as "Max."
answer
['the', 'arms', 'dealer']
question
I'm gonna ask Natalie what the fuck that was all about.
answer
['the', 'one', 'that', 'is', 'about', 'is', 'murder']
question
Never mind.  That is so awful that they hide your cars.
answer
['guess', 'what', 'is', 'truly', 'good', '?']
question
Everything alright, Mr. Boone?
answer
['just', 'got', 'away', 'from', 'me', 'from', 'the', 'sheriff', 'is', 'family']
question
Ten million dollars. Ten mi -- YOU CHEAPSKATES! I've just wiped out the stock market. I've cost you billions!  I want ten million and one.
answer
['we', 'do', 'not', 'know', 'that']
******************************************
question
Ed, I swear, I'm not --
answer
['yes', "ma'am", '?', 'u', 'letters', 'u', 'mine', 'u', 'both', 'of', 'you', 'know', 'who', 'i', 'am']
question
We can

question
... what... ?
answer
['i', 'did', 'not', 'want', 'to', 'come', 'i', 'did', 'not', 'want', 'to', 'sound', 'cynical', 'i', 'knew', 'that', 'was', 'not', 'what', 'i', 'needed', 'but', 'i']
question
Of course.
answer
['and', 'then', '...', '?']
question
Of love?
answer
['yes']
question
Because that's understandable, because. She blew the raid.
answer
['she', 'was', 'there', 'alone', 'sir', 'she', 'was', 'in', 'a', 'burning', 'building', 'waiting', 'for', 'your', 'folks', 'to', 'come', 'through', 'the', 'wall', 'and', '...']
question
Sam, Ferrari wants you to work for him at the Blue Parrot.
answer
['i', 'like', 'it', 'fine', 'here']
******************************************
question
Kalen, right now you need to go up to bed.  You're exhausting Alexander.
answer
['...', 'that', 'is', 'all', 'right', 'i', 'got', 'it', 'but', 'he', 'is', 'not', 'well', 'enough', 'to', 'get', 'in', 'bed', 'with', 'you']
question
Like I say, Doc Long's...
answer
['hell', '...', 'i', 'have', 'seen', 'y

  0%|          | 40/10000 [5:48:00<1453:34:18, 525.39s/it]

question
Where did that come from?
answer
['i', 'have', 'not', 'been', 'out', 'there', 'since', 'the', 'ages', 'ago']
question
You stink, Sefton!
answer
['what', '?']
question
That's all right -- it's no secret. My name's Hartwell. I'm from the company's Eastern office.
answer
['you', 'mean', '?']
question
Sir?
answer
['i', 'am', 'not', 'interested', 'in', 'mythology', 'we', 'can', 'discuss']
question
I think I scared it!
answer
['what', 'are', 'you', 'doing', '?']
******************************************
question
Come on, everybody!  We gotta run for those rocks over there!
answer
['val', 'it', 'is', 'pretty', 'far']
question
The ah... the radio's out.
answer
['what', '?']
question
But...
answer
['i', 'am', 'sorry', 'i', 'am', 'just', '...']
question
Gallagher?
answer
['yes', '?']
question
No, you give me a break! I never left a source hung out to dry, ever. Abandoned. Not 'til right fucking now! When I came on this job, I came with my word intact. I'm gonna leave with my word intac

question
It's too short!  We need more monkeys!
answer
['there', 'are', 'not', 'any', 'more', '!', 'that', 'is', 'the', 'whole', 'barrel']
question
The grooves around the door -- they've shifted.  And the pattern -- the fluids -- they look darker now.
answer
['how', 'is', 'harry', '?']
question
Are you kidding?  Harry had to go out of town.
answer
['i', 'am', 'shocked']
question
Yeah.  You should have seen it. We were the hit of the show with the new model ordering system, the one for the 680... how you can write up an order and the second the customer's name goes into the computer, it starts checking their credit, and if they've ordered anything in the past, and if they get a discount...
answer
['the', 'twenty', 'i', 'have', 'seen', 'them', 'a', 'thousand', 'times', 'at', 'the', 'police', 'side']
question
How about that talent?  This is like falling into a tub of butter.
answer
['watch', 'it', '!']
******************************************
question
Where's the body?
answer
['the', 's

In [19]:
agent.generate_sentences(["How are you doing?"])

[['fine']]

In [20]:
agent.generate_sentences(["What is your name?"])

[[]]

In [21]:
agent.generate_sentences(["You do not have a name?"])

[['lewis']]

In [23]:
agent.generate_sentences(["What the fuck? Why were you silent when asking you for the first time"])

[['i', 'did', 'not', 'get', 'it', '...']]

In [26]:
questions = ["How are you doing?",
             "What is your name?", 
             "You do not have a name?",
             "What the fuck? Why were you silent when asking you for the first time"]
predictions = agent.generate_sentences(questions)
for q, p in zip(questions, predictions):
    print("Me:")
    print(q)
    print("Robot:")
    print(" ".join(p))

Me:
How are you doing?
Robot:
fine
Me:
What is your name?
Robot:

Me:
You do not have a name?
Robot:
lewis
Me:
What the fuck? Why were you silent when asking you for the first time
Robot:
i did not get it ...


In [ ]:
agent.fit(train_X, train_y, num_epochs=10000, batch_bar=False, log_freq=1, batch_log_freq=100)


  0%|          | 0/10000 [00:00<?, ?it/s]

Model saved in file: params/model.ckpt


question
An' she's got a beau! Kisses him, too!
answer
['you', 'thought', 'that', 'was', 'the', 'traffic', 'huh', '?']
question
I bet you do!
answer
['do', 'not', 'you', 'like', 'better', 'than', 'me', '?']
question
I dunno... Cinnamon roll?
answer
['i', 'am', 'no', 'forensic', 'promise', 'you', 'but', 'i', 'am', 'no', 'in', 'fact', 'that', 'looks', 'like', 'i', 'am', 'no', 'forensic', 'expert', 'mind', 'you', 'but']
question
What do you mean - now?
answer
['well', 'i', 'mean', 'i', 'mean', 'i', 'mean', 'i', 'know', 'this', 'is', 'not', 'what', 'i', 'mean']
question
It can't be as bad as that.
answer
['what', 'are', 'you', 'talking', 'about', '?']
******************************************
question
No... they didn't survive only to be your food.  You did that.
answer
['i', 'am', 'afraid', 'your', 'is', 'lost', 'here', 'i', 'have', 'no', 'more', 'human', 'response', 'to', 'the', 'then', 'you', 'would', 'have', 'to', 'a', 'carrot', 'it']
question
Are you sure? How do you know?
answer
['i

question
Yes.
answer
['i', 'am', 'not', 'much', 'of', 'a', 'reader', 'but', 'i', 'do', 'not', 'really', 'like', 'it', 'i', 'was', 'just', 'beginning', 'to', 'see', 'the', 'new', 'work']
question
I'm looking!
answer
['what', 'do', 'you', 'expect', '?']
question
The city?  New York City?  Why?
answer
['because', 'thats', 'where', 'i', 'live', 'and', 'i', 'am', 'going', 'to', 'see', 'what', 'i', 'am', 'going', 'to', 'do']
question
You must have an interesting job at a salary which you would regard as adequate. Not only for the job which you are going to do and in compensation for what you believe you have suffered, but also because you are helping us.
answer
['i', 'have', 'nothing', 'to', 'say', 'to', 'make', 'your', 'decision', 'the', 'general', 'have', 'been', 'done']
question
Taylor, my dear.
answer
['can', 'you', 'get', 'it', '?']
******************************************
question
What's your fucking problem? You're a shithead!
answer
['i', 'just', 'want', 'to', 'make', 'a', 'problem

  0%|          | 1/10000 [08:31<1420:52:11, 511.56s/it]

...

question
He's convulsing!
answer
['it', 'is', 'too', 'much', '!']
question
The girl. The girl has the disc I need.
answer
['i', 'told', 'you', 'i', 'do', 'not', 'play', 'well', 'with', 'others']
question
What's the delay?
answer
['let', 'is', 'move', 'your', 'ass', 'in', 'there', '!', '!']
question
I know...I mean you didn't do anything special for me tonight.  You just had what I think are good manners, decency. And it really makes me want to be nice back and it has nothing to do with any homosexual thing.  Honestly.  Because I don't know if you've homosexual or not and -- you're not, are you?
answer
['i', 'do', 'not', 'know']
question
Maybe you saw Mary!
answer
['no']
******************************************
question
No.  No.
answer
['i', 'am', 'not', 'sure']
question
Why not?
answer
['i', 'do', 'not', 'know']
question
Tyler Durden.
answer
['i', 'am', 'not', 'sure']
question
He has to go, Samuel.
answer
['what', '?', 'who', '?']
question
What?
answer
['i', 'am', 'not', 'sure']
****

question
Even women?
answer
['there', 'are', 'not', 'any', 'women', 'here']
question
Right.  And they know him.  And that's why it would never occur to them for a minute that their old buddy Bruce puts on a cape at night and goes out looking for --
answer
['it', 'is', 'will', 'you', '...', '?']
question
What are we going to do?
answer
['there', 'is', 'nothing', 'we', 'can', 'do', 'until', 'the', 'morning']
question
Fascinating.
answer
['now', 'that', 'i', 'have', 'come', 'to', 'get', 'you', 'to', 'a', 'doctor', 'right', '?', 'well', 'let', 'is', 'say', 'i', 'get', 'it', 'straight']
question
Does anybody know I'm here?
answer
['only', 'the', 'elders']
******************************************
question
Hi.
answer
['hi']
question
They respond pretty quick.  I'll wait till you get down before I turn it on.
answer
['good', 'see', 'ya', 'soon']
question
May I ask Why?
answer
['your', 'dream', 'made', 'me', 'go', 'for', 'a', 'while']
question
I've got the Panasonic Portable DVD player.
answe

  0%|          | 6/10000 [51:45<1436:19:50, 517.39s/it]

question
When - you're a big boy.  I mean you know it's bad for you
answer
['but', 'mom', 'i', 'need', 'the', 'badly']
question
I got out, Nick didn't! I pretended I was him! I knew about her letters! Jesus Christ, whatever you want from him -- I'm not Nick!  I -- I just wanted to be --
answer
['i', 'have', 'been', 'to', 'go', 'into', 'this', 'strange', 'believe', 'me', 'i', 'have', 'so', 'much', 'of', 'all', 'this', 'shit']
question
I could try to...make it stop.
answer
['and', 'so', 'where', 'is', 'the', 'hit', '?']
question
Nothing
answer
['you', 'are', 'not', 'eating']
question
Spock... don't.
answer
['it', 'is', 'the', 'same', 'way', 'in']
******************************************
question
I'm sorry.  I have to bring you to a cell.
answer
['my', 'parents', 'are', 'him', 'what', 'do', 'you', 'mean', '?']
question
The water.
answer
['what', 'about', 'the', 'water', '?']
question
What about the killing and looting? That's just as crude, isn't it?
answer
['the', 'thing', 'is']
questi

question
I mean, I'm not much use anymore. Why can't they be compassionate to me?
answer
['just', 'what', 'you', 'thought', 'about']
question
I won't be back. I've requested the Atlanta run.
answer
['no', 'you', 'can', 'wait']
question
Snickers, baby, I love you like a brother, but really, who cares? Silencer bomb...
answer
['it', 'is', 'she', 'is', 'an', 'adult', 'conversation']
question
Hi, everybody.  Sorry to be late -- had to have dinner with my depart- ment chief --
answer
['you', 'ate', '?']
question
Hey, man. Good to see you. What'll it be today?
answer
['to', 'keep', 'an', 'eye', 'out']
******************************************
question
Asshole!!
answer
['hey', 'what', 'is', 'the', 'matter', '?']
question
M'sieu ...
answer
['yes', 'madame', '?']
question
JUst don't go in there half cocked. You guys have a tendency to blow shit up and ask questions later.
answer
['you', 'got', 'that', 'right']
question
I don't know if you read the papers, but that case is closed.
answer
['i', 